In [158]:
import pandas as pd

# getting the data from the text file and transforming it to a suitable format.

In [159]:
f = open('text.txt', 'r')
data = f.read()
paragraphs = data.split("\n\n")
paragraphs[:] = (value for value in paragraphs if value != '\t')


In [160]:
lis=[]
for i in range(len(paragraphs)):
    lst=[]
    lst.append(paragraphs[i])
    lis.append(lst)
    
    

In [161]:
df = pd.DataFrame(lis)


In [162]:
df.head()

,0
0,"ï»¿From fairest creatures we desire increase,\..."
1,\n
2,"When forty winters shall besiege thy brow,\nAn..."
3,\n
4,Look in thy glass and tell the face thou viewe...


In [163]:
df1=df.iloc[::2]

In [164]:
df2=df1.reset_index()

In [165]:
df2=df2.drop(['index'], axis=1)

In [166]:
X=df2.rename(index=str, columns={0: "text"})

# let's preprocess the text, before doing that let's engineer some features

In [167]:
X['qlen'] = X['text'].str.len()#getting the length of the sonnet. 
   
X['q_n_words'] = X['text'].apply(lambda row: len(row.split(" ")))#the number of words in the sonnet.

In [168]:
X.head()

,text,qlen,q_n_words
0,"ï»¿From fairest creatures we desire increase,\...",628,95
1,"When forty winters shall besiege thy brow,\nAn...",644,104
2,Look in thy glass and tell the face thou viewe...,610,104
3,"Unthrifty loveliness why dost thou spend,\nUpo...",591,93
4,Those hours that with gentle work did frame\nT...,626,93


In [169]:
def avg_word(sentence):#getting the average word length for each sonnet.
    
    words = sentence.split()
    return (sum(len(word) for word in words)/len(words))

X['avg_word'] = X['text'].apply(lambda x: avg_word(x))

In [170]:
from nltk.corpus import stopwords#getting the number of stop words in each sonnet.
stop = stopwords.words('english')

X['stopwords'] = X['text'].apply(lambda x: len([x for x in x.split() if x in stop]))

In [171]:
X.head()

,text,qlen,q_n_words,avg_word,stopwords
0,"ï»¿From fairest creatures we desire increase,\...",628,95,4.915094,28
1,"When forty winters shall besiege thy brow,\nAn...",644,104,4.591304,28
2,Look in thy glass and tell the face thou viewe...,610,104,4.295652,42
3,"Unthrifty loveliness why dost thou spend,\nUpo...",591,93,4.673077,23
4,Those hours that with gentle work did frame\nT...,626,93,5.009615,32


# now let's preprocess and clean the text data

In [172]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from subprocess import check_output
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import os
import gc

import re
from nltk.corpus import stopwords
import distance
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
# This package is used for finding longest common subsequence between two strings
# you can write your own dp code for this
import distance
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
from sklearn.manifold import TSNE
# Import the Required lib packages for WORD-Cloud generation
# https://stackoverflow.com/questions/45625434/how-to-install-wordcloud-in-python3-6
from wordcloud import WordCloud, STOPWORDS
from os import path
from PIL import Image

Now that we have finished deduplication our data requires some preprocessing before we go on further with analysis and making the prediction model.

Hence in the Preprocessing phase we do the following in the order below:-

Begin by removing the html tags
Remove any punctuations or limited set of special characters like , or . or # etc.
Check if the word is made up of english letters and is not alpha-numeric
Check to see if the length of the word is greater than 2 (as it was researched that there is no adjective in 2-letters)
Convert the word to lowercase
Remove Stopwords
Finally Snowball Stemming the word (it was obsereved to be better than Porter Stemming)


In [173]:
# To get the results in 4 decemal points
SAFE_DIV = 0.0001 

STOP_WORDS = stopwords.words("english")


def preprocess(x):
    x = str(x).lower()
    x = x.replace(",000,000", "m").replace(",000", "k").replace("′", "'").replace("’", "'")\
                           .replace("won't", "will not").replace("cannot", "can not").replace("can't", "can not")\
                           .replace("n't", " not").replace("what's", "what is").replace("it's", "it is")\
                           .replace("'ve", " have").replace("i'm", "i am").replace("'re", " are")\
                           .replace("he's", "he is").replace("she's", "she is").replace("'s", " own")\
                           .replace("%", " percent ").replace("₹", " rupee ").replace("$", " dollar ")\
                           .replace("€", " euro ").replace("'ll", " will")
    x = re.sub(r"([0-9]+)000000", r"\1m", x)
    x = re.sub(r"([0-9]+)000", r"\1k", x)
    
    
    porter = PorterStemmer()
    pattern = re.compile('\W')
    
    if type(x) == type(''):
        x = re.sub(pattern, ' ', x)
    
    
    if type(x) == type(''):
        x = porter.stem(x)
        example1 = BeautifulSoup(x)
        x = example1.get_text()
               
    
    return x
    

In [174]:
X["text"] = X["text"].fillna("").apply(preprocess)

# 1)first assignment(tag each sonnet with the most frequent word and the most important word)

# getting the most frequent word for each sonnet

In [175]:
X.head()

,text,qlen,q_n_words,avg_word,stopwords
0,ï from fairest creatures we desire increase ...,628,95,4.915094,28
1,when forty winters shall besiege thy brow and...,644,104,4.591304,28
2,look in thy glass and tell the face thou viewe...,610,104,4.295652,42
3,unthrifty loveliness why dost thou spend upon...,591,93,4.673077,23
4,those hours that with gentle work did frame th...,626,93,5.009615,32


In [176]:
a=X.iloc[3]
print(X['text'].iloc[1])

when forty winters shall besiege thy brow  and dig deep trenches in thy beautyâ euro  s field  thy youthâ euro  s proud livery so gazed on now  will be a tattered weed of small worth held  then being asked  where all thy beauty lies  where all the treasure of thy lusty days  to say  within thine own deep sunken eyes  were an all eating shame  and thriftless praise  how much more praise deservâ euro  d thy beautyâ euro  s use  if thou couldst answer â euro  this fair child of mine shall sum my count  and make my old excuse â euro   proving his beauty by succession thine   this were to be new made when thou art old   and see thy blood warm when thou feelâ euro  st it cold 


In [177]:
from collections import Counter#getting the most occuring string.
fre=[]
for i in range(len(X)):
    s=X['text'].iloc[i]
    s=str(s)
    split_it = s.split() 
  
    # Pass the split_it list to instance of Counter class. 
    Counter1 = Counter(split_it) 
  
    # most_common() produces k frequently encountered 
    # input values and their respective counts. 
    most_occur = Counter1.most_common(1) 
  
    fre.append(most_occur)

In [178]:
wo=[]
for i in range(len(fre)):
    
    w=fre[i]
    wor=w[0][0]
    wo.append(wor)
    
    
    
    

In [179]:
X['most_frequent_word'] = wo

# 2)recommend the similar sonnets

# to extract the features for text, i am going to train a word to vec model with the shakespeare package from nltk corpus_

In [180]:
from nltk.corpus import gutenberg

In [181]:
gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [182]:
shake_words = gutenberg.words('shakespeare-caesar.txt')

In [183]:
bible_kjv_words

['[', 'The', 'Tragedie', 'of', 'Julius', 'Caesar', ...]

In [184]:
shake_sent = gutenberg.sents('shakespeare-caesar.txt') 

In [185]:
shake_sents

[['[', 'The', 'Tragedie', 'of', 'Julius', 'Caesar', 'by', 'William', 'Shakespeare', '1599', ']'], ['Actus', 'Primus', '.'], ...]

In [186]:
from gensim.models import Word2Vec

In [187]:
shake_sent2=gutenberg.sents('shakespeare-hamlet.txt') 

In [188]:
model = Word2Vec(shake_sent, min_count=1)

In [189]:
words = model.wv.vocab

# now we will extract the text features using the average word to vec model

In [190]:
# vocab = stores all the words that are there in google w2v model
# vocab = model.wv.vocab.keys() # if you are using Google word2Vec

vocab = words.keys()
# this function will add the vectors of each word and returns the avg vector of given sentance
def build_avg_vec(sentence, num_features, doc_id, m_name):
    # sentace: its title of the apparel
    # num_features: the lenght of word2vec vector, its values = 300
    # m_name: model information it will take two values
        # if  m_name == 'avg', we will append the model[i], w2v representation of word i
        # if m_name == 'weighted', we will multiply each w2v[word] with the idf(word)

    featureVec = np.zeros((num_features,), dtype="float32")
    # we will intialize a vector of size 300 with all zeros
    # we add each word2vec(wordi) to this fetureVec
    nwords = 0
    
    for word in sentence.split():
        nwords += 1
        if word in vocab:
            if m_name == 'weighted' and word in  idf_title_vectorizer.vocabulary_:
                featureVec = np.add(featureVec, idf_title_features[doc_id, idf_title_vectorizer.vocabulary_[word]] * model[word])
            elif m_name == 'avg':
                featureVec = np.add(featureVec, model[word])
    if(nwords>0):
        featureVec = np.divide(featureVec, nwords)
    # returns the avg vector of given sentance, its of shape (1, 300)
    return featureVec

In [191]:
from sklearn.metrics import pairwise_distances
doc_id = 0

w2v_title = []
# for every title we build a avg vector representation
for i in X['text']:
    w2v_title.append(build_avg_vec(i, 100, doc_id,'avg'))
    doc_id += 1

# w2v_title = np.array(# number of doc in courpus * 300), each row corresponds to a doc 
w2v_title = np.array(w2v_title)

# let's compute the pair wise distances, in other words euclidean distance,the distance between ith vector and jth vector , and we will return the top 5 distances


In [192]:
def avg_w2v_model(doc_id, num_results):
    # doc_id: apparel's id in given corpus
    
    # dist(x, y) = sqrt(dot(x, x) - 2 * dot(x, y) + dot(y, y))
    pairwise_dist = pairwise_distances(w2v_title, w2v_title[doc_id].reshape(1,-1))

    # np.argsort will return indices of 9 smallest distances
    indices = np.argsort(pairwise_dist.flatten())[0:num_results]
    #pdists will store the 9 smallest distances
    pdists  = np.sort(pairwise_dist.flatten())[0:num_results]

    #data frame indices of the 9 smallest distace's
    df_indices = list(X.index[indices])
    df_i=map(int, df_indices)
    tex=X['text']
    se=[]
    for i in df_i:
        s=tex.iloc[i]
        se.append([i,s])
    return se
    



# now given an index of any sonnet, it returns 5 similar sonnets, with the index number

In [193]:
avg_w2v_model(0, 5)

[[0,
  'ï  from fairest creatures we desire increase  that thereby beautyâ euro  s rose might never die  but as the riper should by time decease  his tender heir might bear his memory  but thou contracted to thine own bright eyes  feedâ euro  st thy lightâ euro  s flame with self substantial fuel  making a famine where abundance lies  thy self thy foe  to thy sweet self too cruel  thou that art now the worldâ euro  s fresh ornament  and only herald to the gaudy spring  within thine own bud buriest thy content  and  tender churl  makâ euro  st waste in niggarding   pity the world  or else this glutton be   to eat the worldâ euro  s due  by the grave and thee '],
 [123,
  'if my dear love were but the child of state  it might for fortuneâ euro  s bastard be unfathered  as subject to timeâ euro  s love or to timeâ euro  s hate  weeds among weeds  or flowers with flowers gathered  no it was builded far from accident  it suffers not in smiling pomp  nor falls under the blow of thralled disc

# 3) getting the most important words

In [194]:
import math
from textblob import TextBlob as tb

def tf(word, blob):
    return blob.words.count(word) / len(blob.words)

def n_containing(word, bloblist):
    return sum(1 for blob in bloblist if word in blob.words)

def idf(word, bloblist):
    return math.log(len(bloblist) / (1 + n_containing(word, bloblist)))

def tfidf(word, blob, bloblist):
    return tf(word, blob) * idf(word, bloblist)

In [198]:
ls=[]
for i in range(len(X['text'])):
    d=X['text'].iloc[i]
    ds=tb(d)
    bloblist = [ds]
    for i, blob in enumerate(bloblist):
        li=[]
        scores = {word: tfidf(word, blob, bloblist) for word in blob.words}
        sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        for word, score in sorted_words[:1]:
            w=word
            s=round(score,5)
            li.append([w,s])
            ls.append(li)
            
            
            

In [200]:
len(ls)

154

In [201]:
X.head()

,text,qlen,q_n_words,avg_word,stopwords,most_frequent_word
0,ï from fairest creatures we desire increase ...,628,95,4.915094,28,euro
1,when forty winters shall besiege thy brow and...,644,104,4.591304,28,thy
2,look in thy glass and tell the face thou viewe...,610,104,4.295652,42,the
3,unthrifty loveliness why dost thou spend upon...,591,93,4.673077,23,thy
4,those hours that with gentle work did frame th...,626,93,5.009615,32,with


In [202]:
X['most_important_word'] = ls

# this will return the most important word and it's score

In [204]:
X

,text,qlen,q_n_words,avg_word,stopwords,most_frequent_word,most_important_word
0,ï from fairest creatures we desire increase ...,628,95,4.915094,28,euro,"[[ï, -0.00578]]"
1,when forty winters shall besiege thy brow and...,644,104,4.591304,28,thy,"[[forty, -0.00533]]"
2,look in thy glass and tell the face thou viewe...,610,104,4.295652,42,the,"[[look, -0.00587]]"
3,unthrifty loveliness why dost thou spend upon...,591,93,4.673077,23,thy,"[[unthrifty, -0.00636]]"
4,those hours that with gentle work did frame th...,626,93,5.009615,32,with,"[[those, -0.00619]]"
5,then let not winterâ euro s ragged hand defac...,620,99,4.627273,28,thou,"[[let, -0.00578]]"
6,lo in the orient when the gracious light lifts...,581,90,4.742574,32,his,"[[lo, -0.00666]]"
7,music to hear why hearâ euro st thou music s...,656,99,4.954545,30,euro,"[[hear, -0.00568]]"
8,is it for fear to wet a widowâ euro s eye th...,631,107,4.338983,34,euro,"[[is, -0.00533]]"
9,for shame deny that thou bearâ euro st love t...,629,104,4.460870,32,that,"[[shame, -0.0055]]"
